In [1]:
import os
import sys
import math
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

import matplotlib
import matplotlib.ticker as ticker
from matplotlib import cm  #to get python's normal library of colormaps
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")  #hides "MatplotlibDeprecationWarning" with pcolormesh

In [2]:
#set some baseline plot displays

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 35
matplotlib.rcParams['axes.titlesize'] = 45
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 27
matplotlib.rcParams['ytick.labelsize'] = 27
matplotlib.rcParams['legend.fontsize'] = 35
#matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'


In [3]:
full = False    #do the chosen time ranges below encapsulate the entire convective module?
case_list = [['20220906', '20220906,110000', '20220906,120000'],
            ['20220906', '20220906,133000', '20220906,161000'],
            ['20220906', '20220906,161000', '20220906,180000'],
            ['20220907', '20220907,130000', '20220907,134500'],
            ['20220907', '20220907,134500', '20220907,161800'],
            ['20220907', '20220907,161800', '20220907,174500'],
            ['20220909', '20220909,161000', '20220909,171000'],
            ['20220909', '20220909,173500', '20220909,191000'],
            ['20220910', '20220910,153500', '20220910,180500'],
            ['20220910', '20220910,180500', '20220910,204500'],
            ['20220914', '20220914,101000', '20220914,115500'],
            ['20220914', '20220914,115500', '20220914,142800'],
            ['20220914', '20220914,142800', '20220914,164500'],
            ['20220916', '20220916,143000', '20220916,164000'],
            ['20220916', '20220916,164000', '20220916,183500'],
            ['20220920', '20220920,071000', '20220920,090000'],
            ['20220922', '20220922,054000', '20220922,083000'],
            ['20220923', '20220923,092000', '20220923,115000'],
            ['20220923', '20220923,115000', '20220923,143000'],
            ['20220926', '20220926,072000', '20220926,092000'],
            ['20220926', '20220926,092000', '20220926,111500'],
            ['20220929', '20220929,103500', '20220929,120000'],
            ['20220929', '20220929,120000', '20220929,134500'],
            ['20220930', '20220930,092000', '20220930,102000'],
            ['20220930', '20220930,111000', '20220930,134000'],
            ['20220930', '20220930,134000', '20220930,143000']]

for start_list in case_list:
    
    print ('Working on:', start_list)
    file_date = start_list[0]    #which date to create the APR plot for
    start_time0 = start_list[1]  #start date,time for the APR plot
    end_time0 = start_list[2]    #end date,time for the APR plot

    #other users will also have to change day_folder, apr_folder, dawn_csv_path, and drop_csv_path filepaths
        #dawn_csv_path and drop_csv_path are based on created files from CPEXCV_DAWN.ipynb and CPEXCV_dropsonde(_metrics).ipynb

    start_time = datetime.strptime(start_time0, '%Y%m%d,%H%M%S')
    end_time = datetime.strptime(end_time0, '%Y%m%d,%H%M%S')

    #locations of the APR folder and APR files
    day_folder = os.path.join(os.getcwd(), file_date)
    apr_folder = os.path.join(day_folder, 'APR_files')

    #load the final DAWN and final Dropsonde CSVs
    dawn_csv_path = os.path.join(day_folder, 'final_dawn_' + file_date + '.csv')
    dawn_csv = pd.read_csv(dawn_csv_path)

    drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
    drop_csv = pd.read_csv(drop_csv_path)

    #create a list of all the given day's desired range's APR files:
    for x in os.listdir(apr_folder):
        if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
            os.remove(os.path.join(apr_folder, x))

    #find the starting APR file in apr_folder
    first_file_index = None 

    #sorted() makes sure the code goes through the files in alphabetical (chronological) order
    #GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
    for i, x in enumerate(sorted(os.listdir(apr_folder))):  
        file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
        file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')

        if start_time <= file_start_time:  #if start_time is before the APR file start time and not within any previous APR file's time ranges
            first_file_index = i
            break
        elif (start_time >= file_start_time) and (start_time < file_end_time):
            first_file_index = i
            break
        else:
            continue
    if first_file_index == None:
        sys.exit('Requested start_time is beyond all available APR files')

    #find the ending APR file in apr_folder
    last_file_index = None    

    #sorted() makes sure the code goes through the files in alphabetical (chronological) order
    #GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
    for i, x in enumerate(sorted(os.listdir(apr_folder))):  
        file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
        file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')

        if end_time <= file_start_time:  #if end_time is before the APR file start time and not within any previous APR file's time ranges
            last_file_index = i - 1
            break
        elif (end_time > file_start_time) and (end_time <= file_end_time):
            last_file_index = i
            break
        else:
            continue
    if last_file_index == None:  #the end_time is after all available APR file ranges
        last_file_index = len(sorted(os.listdir(apr_folder))) - 1  #the last available APR file's index
    if last_file_index == -1:
        sys.exit('Requested end_time is before all available APR files')

    apr_file_list = sorted(os.listdir(apr_folder))[first_file_index:last_file_index + 1]
    #apr_file_list

    #Calculate time range in minutes, have a tick for each minute
    num_minutes = (end_time - start_time).total_seconds() // 60
    fig_length = num_minutes * 2

    #Calculate height range of the plot, based on the maximum height of all DAWN profiles
    height_max = dawn_csv['Height [m]'].max()
    y_max = math.ceil(height_max / 1000) * 1000  #round height_max up to the nearest 1000

    #plotting a time-series plot of Ku-band reflectivity vs. height
    ylim=[0,y_max]
    vlim=[-10,40]
    vlim2=[-15,15]   #for Ku-band minus Ka-band reflectivity
    vel_lim=[-10,10]

    #Low resolution ('lores') radar variables in APR hdf files
    ku_band = 'lores_zhh14' #Ku-band reflectivity
    ka_band = 'lores_zhh35' #Ka-band reflectivity
    vel = 'lores_vel14c' #Mean Doppler Velocity from Ku-band (surface Doppler velocity is subtracted and free of aliasing)
    w_band = 'lores_z95s' #W-band reflectivity
    w_band_nadir = 'hi2lo_z95n' #W-band reflectivity at nadir, upscaled to same resolution as 'lores'
        #^^^for quantitative analysis, will likely want to switch to using 'hires_z95n'

    if full:
        plot_save_name = file_date + '_FivePanel_full.png'
    else:
        plot_save_name = file_date + '_FivePanel_' + start_time0[-6:] + '-' + end_time0[-6:] + '.png'

    fig,axes = plt.subplots(5,1,figsize=(fig_length,60))

    for file in apr_file_list:

        #find the APR file of interest (for the desired datetime)
        apr_filepath = os.path.join(apr_folder, file)
        apr_file = xr.open_dataset(apr_filepath)

        #grab the radar variables of interest
        ku_good = False
        if ku_band in apr_file.keys():
            try:   #some APR files, at least in the preliminary data, have corrupted Ku-band data; 
                   #if so, skip plotting the Ku-band for that file
                   #corrupted: "OSError: Can't read data (inflate() failed)"
                ku_data = apr_file[ku_band][:]

                #mask the missing Ku-band data (values of -99.99)
                ku_masked = np.ma.masked_where(ku_data <= -99, ku_data)
                ku_masked = np.ma.masked_where(np.isnan(ku_masked), ku_masked)  #masks NaN values (not masked in previous line)
                ku_good = True
            except:
                ku_good = False

        ka_good = False
        if ka_band in apr_file.keys():
            try:   #some APR files, at least in the preliminary data, have corrupted Ka-band data; 
                   #if so, skip plotting the Ka-band for that file
                   #corrupted: "OSError: Can't read data (inflate() failed)"
                ka_data = apr_file[ka_band][:]

                #mask the missing Ka-band data (values of -99.99)
                ka_masked = np.ma.masked_where(ka_data <= -99, ka_data)
                ka_masked = np.ma.masked_where(np.isnan(ka_masked), ka_masked)  #masks NaN values (not masked in previous line)
                ka_good = True
            except:
                ka_good = False

        w_nadir_good = False
        if w_band_nadir in apr_file.keys():  #nadir-only W-band data
            try:   #some APR files, at least in the preliminary data, have corrupted W-band data; 
                   #if so, skip plotting the W-band for that file
                   #corrupted: "OSError: Can't read data (inflate() failed)"
                w_nadir_data = apr_file[w_band_nadir][:]

                #mask the missing W-band data (values of -99.99)
                w_nadir_masked = np.ma.masked_where(w_nadir_data <= -99, w_nadir_data)
                w_nadir_masked = np.ma.masked_where(np.isnan(w_nadir_masked), w_nadir_masked)  #masks NaN values (not masked in previous line)
                w_nadir_good = True
            except:
                w_nadir_good = False

        w_good = False
        if w_band in apr_file.keys():
            try:   #some APR files, at least in the preliminary data, have corrupted W-band data; 
                   #if so, skip plotting the W-band for that file
                   #corrupted: "OSError: Can't read data (inflate() failed)"
                w_data = apr_file[w_band][:]

                #mask the missing W-band data (values of -99.99)
                w_masked = np.ma.masked_where(w_data <= -99, w_data)
                w_masked = np.ma.masked_where(np.isnan(w_masked), w_masked)  #masks NaN values (not masked in previous line)
                w_good = True
            except:
                w_good = False

        vel_good = False
        if vel in apr_file.keys():
            try:   #some APR files, at least in the preliminary data, have corrupted Velocity data; 
                   #if so, skip plotting velocity for that file
                   #corrupted: "OSError: Can't read data (inflate() failed)"
                vel_data = apr_file[vel][:]

                #mask the missing Velocity data (values < -30 (changed from original -99.99))
                vel_masked = np.ma.masked_where(vel_data <= -99, vel_data)
                vel_masked = np.ma.masked_where(np.isnan(vel_masked), vel_masked)  #masks NaN values (not masked in previous line)
                vel_good = True
            except:
                vel_good = False 

        #if Ku-band or Ka-band or W-band or Doppler Velocity is available
        if ku_good or ka_good or w_nadir_good or w_good or vel_good:            

            #Convert 'lo-res' APR times to datetimes
            time = apr_file['time'][:]  #For 'lores': Time of scan, in seconds since midnight UTC of [YYYY-mm-DD]
            alt3d = apr_file['lores_alt3D'][:]

            time_dates = np.empty(time.shape, dtype=object)
            for i in np.arange(0, time.shape[0]):
                #hour, second automatically revert to midnight (hour = 0, seconds = 0) for '%Y%m%d'
                time_dates[i] = datetime.strptime(file_date, '%Y%m%d') + timedelta(seconds = float(time[i].values))

            #Create a time at each gate (assuming time is the same for each ray of a given scan and down each ray)      
            time_gate = np.empty(alt3d.shape, dtype=object)
            for i in np.arange(0, alt3d.shape[0]):
                time_gate[i,:,:] = time_dates[i]   #assign the same time to all of a given scan's rays and height bins     

            time3d = np.copy(time_gate)

    #         print (time3d[0,0,:])  #times should be the same
    #         print ('')
    #         print (time3d[0,:,0])  #times should be the same
    #         print ('')
    #         print (time3d[:,0,0])  #times should be different
    #         sys.exit()

            #plot the APR data factoring in the aircraft roll (ray adjustment)
            #choose the "pseudo-nadir" ray factoring in aircraft roll
            roll = apr_file['lores_roll'][:]
            ray_angles = np.arange(-25,25.01,25/12)  #in degrees; 25 rays for each scan, with the middle (13th) scan being zero degrees
            for scan in range(roll.shape[0]):
                if (time_dates[scan] >= start_time) and (time_dates[scan] <= end_time):
                    ac_roll = np.nanmean(roll[scan,:])  #roll varies slightly w/ray, so take the average roll value for a given scan and use that for ray adjustment
                    ray_use = np.argmin(np.abs(ray_angles - ac_roll))  #the index of the ray whose angle is closest to that of ac_roll    

                    #scan + 2 (and not scan + 1) is needed because pcolormesh colors the grid cell from the  
                    #grid cell's time to the subsequent grid cell's time.  If a subsequent grid cell does not exist,  
                    #then pcolormesh cannot/doesn't color the grid cell (remember, slicing is right side EXCLUSIVE, 
                    #so scan:scan + 1 is only 1 element and thus doesn't have a subsequent cell!)
                    #by this same logic, scan:scan + 2 will only color one grid cell, since the 2nd (and last) 
                    #element/grid cell doesn't have a subsequent grid cell

                    if ku_good:
                        pm0 = axes[0].pcolormesh(time3d[scan:scan+2,ray_use,:], alt3d[scan:scan+2,ray_use,:],
                                                 ku_masked[scan:scan+2,ray_use,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
                    if ka_good:
                        pm1 = axes[1].pcolormesh(time3d[scan:scan+2,ray_use,:], alt3d[scan:scan+2,ray_use,:],
                                                 ka_masked[scan:scan+2,ray_use,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
                    if vel_good:
                        pm2 = axes[2].pcolormesh(time3d[scan:scan+2,ray_use,:], alt3d[scan:scan+2,ray_use,:],
                                                 vel_masked[scan:scan+2,ray_use,:], cmap=cm.seismic,vmin=vel_lim[0],vmax=vel_lim[1])
                    if ku_good and ka_good:
                        pm4 = axes[4].pcolormesh(time3d[scan:scan+2,ray_use,:], alt3d[scan:scan+2,ray_use,:],
                                                 (ku_masked - ka_masked)[scan:scan+2,ray_use,:], cmap=cm.seismic,vmin=vlim2[0],vmax=vlim2[1])

                    #prioritize plotting the 'lores' W-band over the 'hires'/'hi2lo' nadir W-band    
                    if w_good:
                        pm3 = axes[3].pcolormesh(time3d[scan:scan+2,ray_use,:], alt3d[scan:scan+2,ray_use,:],
                                                 w_masked[scan:scan+2,ray_use,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
                    elif w_nadir_good:
                        alt3d_nadir = apr_file['hi2lo_alt3D'][:]
                        try:
                            pm3 = axes[3].pcolormesh(time3d[scan:scan+2,12,:], alt3d_nadir[scan:scan+2,12,:], 
                                                     w_nadir_masked[scan:scan+2,12,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
                        except:  #if alt3d_nadir has no height data for the given scan, then just use alt3d which is essentially the same with only extremely minor height differences (if any)
                            pm3 = axes[3].pcolormesh(time3d[scan:scan+2,12,:], alt3d[scan:scan+2,12,:], 
                                                     w_nadir_masked[scan:scan+2,12,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
                        #except ValueError:
                            #print ('For', time3d[scan,12,0], 'x and y arguments to pcolormesh cannot have non-finite values or be of type numpy.ma.core.MaskedArray with masked values')
                            ##the weird error above is caused by "_Wn.nc" files I believe
                    else:
                        pass

#                     #using .scatter() instead of pcolormesh more than doubles the file size
#                     #if you want to keep the file size similar to pcolormesh, use Image.convert (last section of code)
#                     if ku_good:
#                         pm0 = axes[0].scatter(time3d[scan,ray_use,:], alt3d[scan,ray_use,:],
#                                                  c = ku_masked[scan,ray_use,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
#                     if ka_good:
#                         pm1 = axes[1].scatter(time3d[scan,ray_use,:], alt3d[scan,ray_use,:],
#                                                  c = ka_masked[scan,ray_use,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
#                     if vel_good:
#                         pm2 = axes[2].scatter(time3d[scan,ray_use,:], alt3d[scan,ray_use,:],
#                                                  c = vel_masked[scan,ray_use,:], cmap=cm.seismic,vmin=vel_lim[0],vmax=vel_lim[1])
#                     if ku_good and ka_good:
#                         pm4 = axes[4].scatter(time3d[scan,ray_use,:], alt3d[scan,ray_use,:],
#                                                  (ku_masked - ka_masked)[scan,ray_use,:], cmap=cm.seismic,vmin=vlim2[0],vmax=vlim2[1])

#                     #prioritize plotting the 'lores' W-band over the 'hires'/'hi2lo' nadir W-band
#                     if w_good:
#                         pm3 = axes[3].scatter(time3d[scan,ray_use,:], alt3d[scan,ray_use,:],
#                                                  c = w_masked[scan,ray_use,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
#                     elif w_nadir_good:
#                         alt3d_nadir = apr_file['hi2lo_alt3D'][:]
#                         try:
#                             pm3 = axes[3].scatter(time3d[scan,12,:], alt3d_nadir[scan,12,:],
#                                                      c = w_nadir_masked[scan,12,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
#                         except:  #if alt3d_nadir has no height data for the given scan, then just use alt3d which is essentially the same with only extremely minor height differences (if any)
#                             pm3 = axes[3].pcolormesh(time3d[scan:scan+2,12,:], alt3d[scan:scan+2,12,:], 
#                                                      w_nadir_masked[scan:scan+2,12,:], cmap=cm.Spectral_r,vmin=vlim[0],vmax=vlim[1])
#                         #except ValueError:
#                             #print ('For', time3d[scan,12,0], 'x and y arguments to pcolormesh cannot have non-finite values or be of type numpy.ma.core.MaskedArray with masked values')
#                             ##the weird error above is caused by "_Wn.nc" files I believe
#                     else:
#                         pass

        apr_file.close()

    try:
        cbar0 = plt.colorbar(pm0, ax=axes[0], pad = 0.01)
        cbar0.set_label('Z [dBZ]')
    except NameError:  #the desired time range didn't have any good Ku-band data
        pass
    axes[0].set_title('Ku-band Reflectivity')

    try:
        cbar1 = plt.colorbar(pm1, ax=axes[1], pad = 0.01)
        cbar1.set_label('Z [dBZ]')
    except NameError:  #the desired time range didn't have any good Ka-band data
        pass
    axes[1].set_title('Ka-band Reflectivity')

    try:
        cbar2 = plt.colorbar(pm2, ax=axes[2], pad = 0.01)
        cbar2.set_label('Doppler Velocity [m s$^{-1}$]')
    except NameError:  #the desired time range didn't have any good Doppler velocity data
        pass
    axes[2].set_title('Doppler Velocity')

    try:
        cbar3 = plt.colorbar(pm3, ax=axes[3], pad = 0.01)
        cbar3.set_label('Z [dBZ]')
    except NameError:  #the desired time range didn't have any good W-band data
        pass
    axes[3].set_title('W-band Reflectivity')
    
    try:
        cbar4 = plt.colorbar(pm4, ax=axes[4], pad = 0.01)
        cbar4.set_label('[dB]')
    except NameError:  #the desired time range didn't have any good Ku- AND Ka-band data
        pass
    axes[4].set_title('Ku- minus Ka- Reflectivity')

    dawn_skip = slice(None,None,7)   #only plot every 7th DAWN value
    drop_skip = slice(None,None,20)  #only plot every 20th dropsonde value

    #set the plot start time as the beginning time of the first APR file and the
        #plot end time as the end time of the last APR file:
    # range_start = datetime.strptime(apr_file_list[0][32:40] + apr_file_list[0][41:47], '%Y%m%d%H%M%S') 
    # range_end = datetime.strptime(apr_file_list[-1][49:57] + apr_file_list[-1][58:64], '%Y%m%d%H%M%S')
    range_start = start_time
    range_end = end_time

    for i in range(5):
        axes[i].barbs(pd.to_datetime(dawn_csv['Time [UTC]'])[dawn_skip], dawn_csv['Height [m]'][dawn_skip], 
                      dawn_csv['U Comp of Wind [m/s]'][dawn_skip], dawn_csv['V Comp of Wind [m/s]'][dawn_skip], 
                      fill_empty = True, length = 7, pivot='middle', sizes=dict(emptybarb=0.075), barbcolor = 'k')
        axes[i].barbs(pd.to_datetime(drop_csv['Time [UTC]'])[drop_skip], drop_csv['Height [m]'][drop_skip], 
                      drop_csv['U Comp of Wind [m/s]'][drop_skip], drop_csv['V Comp of Wind [m/s]'][drop_skip], 
                      fill_empty = True, length = 7, pivot='middle', sizes=dict(emptybarb=0.075), barbcolor = 'b')
        axes[i].set_ylabel('Altitude [m]')
        axes[i].set_xlabel('Time [UTC]')
        axes[i].set_ylim([ylim[0],ylim[1]])
        axes[i].tick_params(axis='x', rotation = 50)
        axes[i].tick_params(length = 15, width = 5)
        axes[i].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H:%M:%S"))
        axes[i].xaxis.set_major_locator(ticker.MaxNLocator(num_minutes))      #sets number of ticks
        axes[i].set_xlim([np.datetime64(range_start),np.datetime64(range_end)])    
            #use the above line to narrow the plot's time range (even within a file!!)
                #range_start and range_end must be a datetime object or a string with the 
                    #format: 'YYYY-MM-DD HH:MM:SS' or 'YYYY-MM-DDTHH:MM:SS'

    plt.tight_layout()
    fig.subplots_adjust(hspace=0.40)

    #save the figure
    plt.savefig(os.path.join(day_folder, plot_save_name), bbox_inches = 'tight')
    #plt.show()  #if you want to also show the image in the output cell, plt.show() must come after plt.savefig() in order for the image to save properly
    plt.close()


    ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
    ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
    from PIL import Image
    im = Image.open(os.path.join(day_folder, plot_save_name))

    try:
        im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
    except:
        #use this for older version of PIL/Pillow if the above line doesn't work, 
        #though this line will have isolated, extremely minor image effects due to 
        #only using 256 colors instead of the 3-element RGB scale
        im2 = im.convert('P')

    im2.save(os.path.join(day_folder, plot_save_name))
    im.close()
    im2.close()
    
    print ('Finished:', start_list)
    print ('')


Working on: ['20220910', '20220910,153500', '20220910,180500']
Finished: ['20220910', '20220910,153500', '20220910,180500']

Working on: ['20220910', '20220910,180500', '20220910,204500']
Finished: ['20220910', '20220910,180500', '20220910,204500']

Working on: ['20220914', '20220914,101000', '20220914,115500']
Finished: ['20220914', '20220914,101000', '20220914,115500']

Working on: ['20220914', '20220914,115500', '20220914,142800']
Finished: ['20220914', '20220914,115500', '20220914,142800']

Working on: ['20220914', '20220914,142800', '20220914,164500']
Finished: ['20220914', '20220914,142800', '20220914,164500']

Working on: ['20220916', '20220916,143000', '20220916,164000']
Finished: ['20220916', '20220916,143000', '20220916,164000']

Working on: ['20220916', '20220916,164000', '20220916,183500']
Finished: ['20220916', '20220916,164000', '20220916,183500']

Working on: ['20220920', '20220920,071000', '20220920,090000']
Finished: ['20220920', '20220920,071000', '20220920,090000']

